# Récupération des données pour l'étude communale

In [406]:
ignore_warning=True
# ignore_warning=False

## 1- Importation des librairies externes et classiques

In [407]:
# Importation des librairies externes
!pip install git+https://github.com/InseeFrLab/Py-Insee-Data@master --ignore-install PyYAML
!pip install python-Levenshtein
!pip install openpyxl
!pip install fuzzywuzzy

  Cloning https://github.com/InseeFrLab/Py-Insee-Data (to revision master) to /tmp/pip-req-build-x1zwg3yp
  Running command git clone --filter=blob:none -q https://github.com/InseeFrLab/Py-Insee-Data /tmp/pip-req-build-x1zwg3yp
  Resolved https://github.com/InseeFrLab/Py-Insee-Data to commit 52139da402bc33da6b8892d7d3208f1d2ddfaa02
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached PyYAML-6.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (661 kB)
  Using cached pandas-1.3.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.5 MB)
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached DateTime-4.3-py2.py3-none-any.whl (60 kB)
  Using cached Unidecode-1.3.2-py3-none-any.whl (235 kB)
  Using cached pathlib2-2.3.6-py2.py3-none-any.whl (17 kB)
  Using cached re

In [408]:
# Importation des librairies classiques de python
import unicodedata
import requests
import time
import os
from tqdm import tqdm
import io
import warnings

# Importation des librairies de datascience et autres
import pandas as pd
import numpy as np
import pynsee
import pynsee.download
from pynsee.sirene import *
from pynsee.utils.init_conn import init_conn
from zipfile import ZipFile, Path
from fuzzywuzzy import fuzz
from bs4 import BeautifulSoup

In [409]:
# token API insee
init_conn(insee_key=os.environ['INSEE_KEY'], insee_secret=os.environ['INSEE_SECRET'])

## 2- Téléchargement du Code Officiel Géographique

* Pour des raisons de simplicité, nous travaillons en géographie 2021 :

In [410]:
cog_commune = pynsee.download.telechargerDonnees("COG_COMMUNE", date = "dernier")
URL_COG_2021="https://www.insee.fr/fr/statistiques/fichier/5057840/commune2021-csv.zip"

#Télécharger le zip de l'URL
r=requests.get(URL_COG_2021)

open("commune2021-csv.zip", 'wb').write(r.content)
    
    
with ZipFile("commune2021-csv.zip",'r') as myzip:
   data = myzip.open("commune2021.csv")

donnees_cog_2021=pd.read_csv(data,dtype=str)
# On enleve les communes associées,arrondissements et déléguées pour simplifier le dataframe
donnees_cog_2021=donnees_cog_2021[~donnees_cog_2021.TYPECOM.isin(["COMA","COMD","ARM"])]
donnees_cog_2021['NCC et DEP']= donnees_cog_2021['NCC'] + " " + donnees_cog_2021['DEP']
donnees_cog_2021['NCC et COM']= donnees_cog_2021['NCC'] + " ," + donnees_cog_2021['COM']

Data will be stored in the following location: /tmp/tmpq6ylj0rw


Downloading: : 1.93MiB [00:02, 857kiB/s]                          
/opt/conda/lib/python3.9/site-packages/pynsee/download/__init__.py:181: ResourceWarning: unclosed file <_io.BufferedReader name='/tmp/tmpyom_iy3q'>
  if hashlib.md5(open(filename, 'rb').read()).hexdigest() != caract['md5']:
/opt/conda/lib/python3.9/site-packages/pynsee/download/__init__.py:182: UserWarning: File in insee.fr modified or corrupted during download
  warnings.warn("File in insee.fr modified or corrupted during download")
/opt/conda/lib/python3.9/tempfile.py:817: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmpq6ylj0rw'>
  _warnings.warn(warn_message, ResourceWarning)


No destination directory defined. Data have been written there: /tmp/tmpyom_iy3q


Extracting: 100%|██████████| 9.00M/9.00M [00:00<00:00, 138MB/s]
/tmp/ipykernel_1010/2942342963.py:7: ResourceWarning: unclosed file <_io.BufferedWriter name='commune2021-csv.zip'>
  open("commune2021-csv.zip", 'wb').write(r.content)


In [411]:
donnees_cog_2021.describe()

,TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT,NCC et DEP,NCC et COM
count,34965,34965,34965,34965,34965,34948,34965,34965,34965,34965,34849,0,34965,34965
unique,1,34965,18,101,100,332,8,32488,32553,32707,2039,0,34963,34965
top,COM,01001,44,62,62D,621,0,SAINTE COLOMBE,Sainte-Colombe,Sainte-Colombe,5723,NaN,LONGEVILLE 25,"ABERGEMENT CLEMENCIAT ,01001"
freq,34965,1,5121,890,890,357,28696,12,12,12,135,NaN,2,1


* création d'une fonction permettant de trouver une commune à partir d'un libellé (codage/codification) à l'aide de la librairie fuzzywuzzy

In [412]:
def trouve_commune_with_fuzz(libelle,dep):
    mondf=donnees_cog_2021[donnees_cog_2021['DEP']==dep]
    mondf['score']=mondf['NCC'].apply(lambda x: fuzz.token_sort_ratio(x,libelle))
    mondf=mondf.sort_values(by="score",ascending=False)
    return mondf['NCC et COM'].iloc[0]

## 3- Téléchargement des données de SIRENE

In [413]:
### Téléchargement des données Sirene sur les établissements Siret de boucherie-charcuterie (code 4722Z) à l'instant T
# Téléchargement via Insee sirene data  by sirene.fr website

session = requests.session()
URL_ACCUEIL = "https://www.sirene.fr/sirene/public/creation-fichier"
URL_REQUETE_POST = "https://www.sirene.fr/sirene/public/verifier-creation-fichier.action"
URL_DEVIS="https://www.sirene.fr/sirene/client//sirene/client/devis-demande.action"

URL_RECAP="https://www.sirene.fr/sirene/client/telechargement-pret-fichier.action"
URL_TELECHARGEMENT_POST="https://www.sirene.fr/sirene/client/telechargement-fichier"
POST_DATA={
'fichier.caracteristiques.etat.actif': 'true',
'__checkbox_fichier.caracteristiques.etat.actif': 'true',
'__checkbox_fichier.caracteristiques.etat.ferme': 'true',
'fichier.caracteristiques.typeEtablissement': '1',
'fichier.caracteristiques.geoFile': 'false',
'fichier.caracteristiques.cjFile': 'false',
'fichier.caracteristiques.apeFile': 'false',
'fichier.caracteristiques.sirenSiretFile': 'false',
'fichiersCriteres.criteresGeographie.upload': '(binaire)',
'__multiselect_fichier.criteres.geographie.region.modalites': '',
'__multiselect_fichier.criteres.geographie.departement.modalites': '',
'fichiersCriteres.criteresActivite.upload': '(binaire)',
'__multiselect_fichier.criteres.activite.section.modalites': '',
'__multiselect_fichier.criteres.activite.division.modalites': '',
'__multiselect_fichier.criteres.activite.groupe.modalites': '',
'fichier.criteres.activite.sousClasse.modalites': '4722Z',
'__multiselect_fichier.criteres.activite.sousClasse.modalites': '',
'fichiersCriteres.criteresNature.upload': '(binaire)',
'__multiselect_fichier.criteres.nature.entrepriseNiveau1.modalites': '',
'__multiselect_fichier.criteres.nature.entrepriseNiveau2.modalites': '',
'__multiselect_fichier.criteres.nature.entrepriseNiveau3.modalites': '',
'__multiselect_fichier.criteres.trancheEffectif.modalites': '',
'fichier.criteres.dateCreation.date': '',
'fichier.criteres.dateCreation.dateFin': '',
'fichier.criteres.dateMaj.date': '',
'fichier.criteres.dateMaj.dateFin': '',
'fichiersCriteres.criteresSirenSiret.upload': '(binaire)',
'__multiselect_fichier.criteres.criteresCatEnt.modalites': '',
'strutsLocale': 'fr_FR',
}

result=session.get(URL_ACCUEIL)
result=session.post(URL_REQUETE_POST, data=POST_DATA)


result=session.get(URL_RECAP)

result=session.get(URL_DEVIS)

result=session.post(URL_TELECHARGEMENT_POST)

donnees_sirene_fr=pd.read_csv(io.StringIO(result.content.decode('utf-8')),dtype=str)


* les données de Sirene.fr sont des données d'entreprises France entière. Pour l'étude, le nombre d'établissement par commune est considéré :

In [414]:
donnees_sirene_fr_agregees_par_com = donnees_sirene_fr.groupby("codeCommuneEtablissement").size().reset_index(name='NbBoucheriesParCom').set_index("codeCommuneEtablissement")
donnees_sirene_fr_agregees_par_com.describe()

,NbBoucheriesParCom
count,7026.000000
mean,2.860803
std,6.453578
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,179.000000


In [415]:
donnees_sirene_fr_agregees_par_com

,NbBoucheriesParCom
codeCommuneEtablissement,
01004,7
01021,1
01024,1
01025,3
01026,1
...,...
97610,1
97611,4
97615,1


## 4- Téléchargement des données du ministère de l'intérieur

In [416]:
#Téléchargement des données de crimes et délits rapportés déclarés auprès des commissariats 
url_datagouv_min_interieur="https://www.data.gouv.fr/fr/datasets/r/d792092f-b1f7-4180-a367-d043200c1520"

donnees_police_nationale_2018=pd.read_excel(url_datagouv_min_interieur, sheet_name='Services PN 2018')
donnees_gendarmerie_nationale_2018=pd.read_excel(url_datagouv_min_interieur, sheet_name='Services GN 2018' )

del donnees_police_nationale_2018[donnees_police_nationale_2018.columns[0]]
del donnees_gendarmerie_nationale_2018[donnees_gendarmerie_nationale_2018.columns[0]]

url_localisation_pn="https://www.data.gouv.fr/fr/datasets/r/2cb2f356-42b2-4195-a35c-d4e4d986c62b"
url_localisation_gn="https://www.data.gouv.fr/fr/datasets/r/d6a43ef2-d302-4456-90e9-ff2c47cac562"
donnees_localisation_pn=pd.read_csv(url_localisation_pn,sep=';')
donnees_localisation_gn=pd.read_csv(url_localisation_gn,sep=';')



In [417]:
normalMap = {'À': 'A', 'Á': 'A', 'Â': 'A', 'Ã': 'A', 'Ä': 'A',
             'à': 'a', 'á': 'a', 'â': 'a', 'ã': 'a', 'ä': 'a', 'ª': 'A',
             'È': 'E', 'É': 'E', 'Ê': 'E', 'Ë': 'E',
             'è': 'e', 'é': 'e', 'ê': 'e', 'ë': 'e',
             'Í': 'I', 'Ì': 'I', 'Î': 'I', 'Ï': 'I',
             'í': 'i', 'ì': 'i', 'î': 'i', 'ï': 'i',
             'Ò': 'O', 'Ó': 'O', 'Ô': 'O', 'Õ': 'O', 'Ö': 'O',
             'ò': 'o', 'ó': 'o', 'ô': 'o', 'õ': 'o', 'ö': 'o', 'º': 'O',
             'Ù': 'U', 'Ú': 'U', 'Û': 'U', 'Ü': 'U',
             'ù': 'u', 'ú': 'u', 'û': 'u', 'ü': 'u',
             'Ñ': 'N', 'ñ': 'n',
             'Ç': 'C', 'ç': 'c',
             '§': 'S',  '³': '3', '²': '2', '¹': '1',
             'œ':'oe', 'Œ':'OE'}
normalize = str.maketrans(normalMap)
donnees_localisation_pn["service_sans_accent_upper"]=donnees_localisation_pn.service.apply(lambda x : x.translate(normalize).upper())
donnees_localisation_gn["service_sans_accent_upper"]=donnees_localisation_gn.service.apply(lambda x : x.translate(normalize).upper())

## Traitement des données de la Police Nationale

In [418]:
if ignore_warning:
    warnings.filterwarnings('ignore')


donnees_police_nationale_2018_travail=donnees_police_nationale_2018.T
donnees_police_nationale_2018_travail.columns = donnees_police_nationale_2018_travail.iloc[0]
donnees_police_nationale_2018_travail=donnees_police_nationale_2018_travail.iloc[1: , :]

# On retire les données COM par manque d'autres sources pour les autres données
donnees_police_nationale_2018_travail=donnees_police_nationale_2018_travail[donnees_police_nationale_2018_travail.index.str[:2]!='98']

# 1 - Traitements des commissariats

donnees_police_nationale_2018_commissariat_seulement=donnees_police_nationale_2018_travail[donnees_police_nationale_2018_travail['Libellé index \ CSP'].str.contains("CIAT")]


donnees_police_nationale_2018_commissariat_seulement["LIBELLE_A_RECHERCHER"]=donnees_police_nationale_2018_travail['Libellé index \ CSP'].str.replace('CIAT','')
donnees_police_nationale_2018_commissariat_seulement["LIBELLE_A_RECHERCHER"]=donnees_police_nationale_2018_commissariat_seulement["LIBELLE_A_RECHERCHER"].str.replace('CENTRAL','')
donnees_police_nationale_2018_commissariat_seulement["LIBELLE_A_RECHERCHER"]=donnees_police_nationale_2018_commissariat_seulement["LIBELLE_A_RECHERCHER"].str.replace('DE ','')
donnees_police_nationale_2018_commissariat_seulement["LIBELLE_A_RECHERCHER"]=donnees_police_nationale_2018_commissariat_seulement["LIBELLE_A_RECHERCHER"].str.replace('DES ','')
donnees_police_nationale_2018_commissariat_seulement["LIBELLE_A_RECHERCHER"]=donnees_police_nationale_2018_commissariat_seulement["LIBELLE_A_RECHERCHER"].str.replace('DU ','')
donnees_police_nationale_2018_commissariat_seulement["LIBELLE_A_RECHERCHER"]=donnees_police_nationale_2018_commissariat_seulement["LIBELLE_A_RECHERCHER"].str.replace('D\'','')

donnees_police_nationale_2018_commissariat_seulement["DEP"]=np.where(donnees_police_nationale_2018_commissariat_seulement.index.str[:2]=='97', donnees_police_nationale_2018_commissariat_seulement.index.str[:3],donnees_police_nationale_2018_commissariat_seulement.index.str[:2])
donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement.apply(lambda x: trouve_commune_with_fuzz(x['LIBELLE_A_RECHERCHER'], x.DEP), axis=1)

warnings.resetwarnings()

<>:14: DeprecationWarning: invalid escape sequence \ 
<>:17: DeprecationWarning: invalid escape sequence \ 
<>:14: DeprecationWarning: invalid escape sequence \ 
<>:17: DeprecationWarning: invalid escape sequence \ 
/tmp/ipykernel_1010/3511287045.py:14: DeprecationWarning: invalid escape sequence \ 
  donnees_police_nationale_2018_commissariat_seulement=donnees_police_nationale_2018_travail[donnees_police_nationale_2018_travail['Libellé index \ CSP'].str.contains("CIAT")]
/tmp/ipykernel_1010/3511287045.py:17: DeprecationWarning: invalid escape sequence \ 
  donnees_police_nationale_2018_commissariat_seulement["LIBELLE_A_RECHERCHER"]=donnees_police_nationale_2018_travail['Libellé index \ CSP'].str.replace('CIAT','')


In [419]:
#Reprise manuelle
if ignore_warning:
    warnings.filterwarnings('ignore')

donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement["COMMUNES"].replace('SIGONCE ,04206',"DIGNE LES BAINS ,04070")
donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement["COMMUNES"].replace('EYBURIE ,19079',"BRIVE LA GAILLARDE ,19031")
donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement["COMMUNES"].replace('FONTANES ,42096',"FIRMINY ,42095")
donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement["COMMUNES"].replace('GUMIERES ,42107',"SAINT CHAMOND ,42207")
donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement["COMMUNES"].replace('NEUFBOURG ,50371',"CHERBOURG EN COTENTIN ,50129")
donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement["COMMUNES"].replace('BRULEY ,54102',"VAL DE BRIEY ,54099")
donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement["COMMUNES"].replace('BASLIEUX ,54049',"DOMBASLE SUR MEURTHE ,54159")
donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement["COMMUNES"].replace('BOUQUEHAULT ,62161',"TOUQUET PARIS PLAGE ,62826")
donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement["COMMUNES"].replace('CLERMONT ,74078',"THONON LES BAINS ,74281")
donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement["COMMUNES"].replace('MELZ SUR SEINE ,77289',"MELUN ,77288 ")
donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement["COMMUNES"].replace('MONTIGNY SUR LOING ,77312',"MORET LOING ET ORVANNE ,77316")
donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement["COMMUNES"].replace('SAINT RAPHAEL ,83118',"FREJUS ,83061")
donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement["COMMUNES"].replace('SALERNES ,83121',"SEYNE SUR MER ,83126")
donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement["COMMUNES"].replace('DESTORD ,88130',"SAINT DIE DES VOSGES ,88413")
donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement["COMMUNES"].replace('CERNY ,91129',"EVRY ,91228")
donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement["COMMUNES"].replace('CARRIERES SUR SEINE ,78124',"HOUILLES ,78311")
donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement["COMMUNES"].replace('DAGNY ,77151',"LAGNY SUR MARNE ,77243")
donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement["COMMUNES"].replace('MANAS ,26171',"ROMANS SUR ISERE ,26281")
donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement["COMMUNES"].replace('NOEUX LES MINES ,62617',"BARLIN ,62083")
donnees_police_nationale_2018_commissariat_seulement["COMMUNES"]=donnees_police_nationale_2018_commissariat_seulement["COMMUNES"].replace('POINTE A PITRE ,97120',"LES ABYMES, 97101")

# DIGNE	04	SIGONCE ,04206    -> DIGNE LES BAINS ,04070
# BRIVE	19	EYBURIE ,19079	-> BRIVE LA GAILLARDE ,19031
# ONDAINE	42	FONTANES ,42096 -> FIRMINY ,42095
# GIER	42	GUMIERES ,42107  -> SAINT CHAMOND ,42207
# CHERBOURG	50	NEUFBOURG ,50371  -> CHERBOURG EN COTENTIN ,50129
# BRIEY	54	BRULEY ,54102	-> VAL DE BRIEY ,54099
# DOMBASLE	54	BASLIEUX ,54049	-> DOMBASLE SUR MEURTHE ,54159	
# TOUQUET	62	BOUQUEHAULT ,62161	->TOUQUET PARIS PLAGE ,62826
# LEMAN	74	CLERMONT ,74078	->  THONON LES BAINS ,74281
# MELUN VAL SEINE	77  MELZ SUR SEINE ,77289  ->	MELUN ,77288        (il y a aussi 77350	LE MEE SUR SEINE)
# MORET SUR LOING	77	MONTIGNY SUR LOING ,77312		-> MORET LOING ET ORVANNE ,77316
# FREJUS ST RAPHAEL	83	SAINT RAPHAEL ,83118    -> FREJUS ,83061
# LA SEYNE	83	SALERNES ,83121	-> SEYNE SUR MER ,83126
# ST DIE	88	DESTORD ,88130	-> SAINT DIE DES VOSGES ,88413
# EVRY	91	CERNY ,91129 -> EVRY ,91228
# HOUILLES CARRIERES SUR SEINE	78	CARRIERES SUR SEINE ,78124   -> HOUILLES ,78311
# LAGNY	77	DAGNY ,77151   -> LAGNY SUR MARNE ,77243
# ROMANS	26	MANAS ,26171	-> ROMANS SUR ISERE ,26281
# NOEUX LES MINES ,62617 -> "BARLIN ,62083"
# POINTE A PITRE ,97120 -> "LES ABYMES, 97101"

warnings.resetwarnings()

In [420]:
if ignore_warning:
    warnings.filterwarnings('ignore')

donnees_police_nationale_2018_commissariat_seulement["CODE_INSEE"]=donnees_police_nationale_2018_commissariat_seulement["COMMUNES"].str.replace(r".*,","")
# donnees_police_nationale_2018_commissariat_seulement["CODE_INSEE"]=donnees_police_nationale_2018_commissariat_seulement["CODE_INSEE"].str.replace(r"[A-Z][A-Z]+","")
# donnees_police_nationale_2018_commissariat_seulement["CODE_INSEE"]=donnees_police_nationale_2018_commissariat_seulement["CODE_INSEE"].str.replace(r"[A-Z] ","")
donnees_police_nationale_2018_commissariat_seulement["CODE_INSEE"]=donnees_police_nationale_2018_commissariat_seulement["CODE_INSEE"].str.strip()
donnees_police_nationale_2018_commissariat_seulement["IN_LOC_PN"]=donnees_police_nationale_2018_commissariat_seulement["CODE_INSEE"].isin(donnees_localisation_pn["code_commune_insee"])
donnees_police_nationale_2018_commissariat_seulement[~donnees_police_nationale_2018_commissariat_seulement["IN_LOC_PN"]]
# donnees_police_nationale_2018_commissariat_seulement
#donnees_police_nationale_2018_commissariat_seulement[donnees_police_nationale_2018_commissariat_seulement.LIBELLE_A_RECHERCHER.str.contains("melun",case=False)]
#donnees_localisation_pn[donnees_localisation_pn.service.str.contains("DIGNE",case=False)]
#donnees_localisation_pn[donnees_localisation_pn.service.str.contains("BRIVE",case=False)]
#donnees_localisation_pn[donnees_localisation_pn.service.str.contains("ONDAINE",case=False)]
#donnees_localisation_pn[donnees_localisation_pn.service.str.contains("GIER",case=False)]
#donnees_localisation_pn[donnees_localisation_pn.service.str.contains("CHERBOURG",case=False)]
#donnees_localisation_pn[donnees_localisation_pn.service.str.contains("BRIEY",case=False)]
#donnees_localisation_pn[donnees_localisation_pn.service.str.contains("DOMBASLE",case=False)]
#donnees_localisation_pn[donnees_localisation_pn.service.str.contains("TOUQUET",case=False)]
#donnees_localisation_pn[donnees_localisation_pn.service.str.contains("LéMAN",case=False)]
#donnees_localisation_pn[donnees_localisation_pn.service.str.contains("MELUN",case=False)]
#donnees_localisation_pn[donnees_localisation_pn.service.str.contains("LOING",case=False)]
#donnees_localisation_pn[donnees_localisation_pn.service.str.contains("JUS",case=False)]
#donnees_localisation_pn[donnees_localisation_pn.service.str.contains("SEYNE",case=False)]
#donnees_localisation_pn[donnees_localisation_pn.service.str.contains("DIé",case=False)]
#donnees_localisation_pn[donnees_localisation_pn.service.str.contains("EVRY",case=False)]
#donnees_localisation_pn[donnees_localisation_pn.service.str.contains("HOUILLES",case=False)]
#donnees_localisation_pn[donnees_localisation_pn.service.str.contains("LAGNY",case=False)]
#donnees_localisation_pn[donnees_localisation_pn.service.str.contains("ROMANS",case=False)]
#donnees_localisation_pn[donnees_localisation_pn.service.str.contains("PITRE",case=False)]
#donnees_localisation_pn[donnees_localisation_pn.service.str.contains("NOEUX",case=False)]

warnings.resetwarnings()

In [421]:
commissariat_xy=donnees_localisation_pn[["code_commune_insee","geocodage_x_GPS","geocodage_y_GPS"]]
commissariat_xy=commissariat_xy[commissariat_xy["code_commune_insee"].str[:2]!="98"]
commissariat_xy=commissariat_xy.set_index("code_commune_insee")
donnees_police_nationale_2018_commissariat_seulement=donnees_police_nationale_2018_commissariat_seulement.reset_index()
donnees_police_nationale_2018_commissariat_seulement=donnees_police_nationale_2018_commissariat_seulement.set_index("CODE_INSEE")
donnees_police_nationale_2018_commissariat_seulement=donnees_police_nationale_2018_commissariat_seulement.merge(commissariat_xy,how='inner', left_index=True, right_index=True)


In [ ]:
# donnees_police_nationale_2018_autres=donnees_police_nationale_2018_travail[~donnees_police_nationale_2018_travail['Libellé index \ CSP'].str.contains("CIAT")]


# # 2 - Traitements de Paris
# dpn2018_paris=donnees_police_nationale_2018_autres[donnees_police_nationale_2018_autres.index.str.contains('75')]
# dpn2018_paris['COM']="75056"

# # Les données 92-93-94-95 sont communales et départementales : à analyser finement
# dpn2018_92_93_94_95=donnees_police_nationale_2018_autres[
#     donnees_police_nationale_2018_autres.index.str.contains('92')
#     |donnees_police_nationale_2018_autres.index.str.contains('93')
#     |donnees_police_nationale_2018_autres.index.str.contains('94')
#     |donnees_police_nationale_2018_autres.index.str.contains('95')
# ]
# dpn2018_92_93_94_95["COMMUNES"]=dpn2018_92_93_94_95['Libellé index \ CSP'].apply(trouve_commune_with_fuzz)
# dpn2018_92_93_94_95


In [ ]:
# dpn2018_hors_paris=donnees_police_nationale_2018_autres[~donnees_police_nationale_2018_autres.index.str.contains('75')]

# dpn2018_hors_paris[~dpn2018_hors_paris['Libellé index \ CSP'].str.contains("BMR")
#                                      &~dpn2018_hors_paris['Libellé index \ CSP'].str.contains("BCFZ")
#                                     &~dpn2018_hors_paris['Libellé index \ CSP'].str.contains("USG")]

## Traitement des données de la Gendarmerie Nationale

In [423]:
donnees_gendarmerie_nationale_2018_par_com=donnees_gendarmerie_nationale_2018.T
donnees_gendarmerie_nationale_2018_par_com.columns = donnees_gendarmerie_nationale_2018_par_com.iloc[0]
donnees_gendarmerie_nationale_2018_par_com=donnees_gendarmerie_nationale_2018_par_com.iloc[1: , :]
donnees_gendarmerie_nationale_2018_par_com.columns


# On retire les données COM par manque d'autres sources pour les autres données
donnees_gendarmerie_nationale_2018_par_com=donnees_gendarmerie_nationale_2018_par_com[donnees_gendarmerie_nationale_2018_par_com.index.str[:2]!='98']
donnees_gendarmerie_nationale_2018_par_com=donnees_gendarmerie_nationale_2018_par_com[donnees_gendarmerie_nationale_2018_par_com.index.str[:3]!='978']

donnees_gendarmerie_nationale_2018_par_com["LIBELLE_A_RECHERCHER"]=donnees_gendarmerie_nationale_2018_par_com['Libellé index \ CGD'].str.replace('CGD','')

donnees_gendarmerie_nationale_2018_par_com["DEP"]=np.where(donnees_gendarmerie_nationale_2018_par_com.index.str[:2]=='97', donnees_gendarmerie_nationale_2018_par_com.index.str[:3],donnees_gendarmerie_nationale_2018_par_com.index.str[:2])
donnees_gendarmerie_nationale_2018_par_com["COMMUNES"]=donnees_gendarmerie_nationale_2018_par_com.apply(lambda x: trouve_commune_with_fuzz(x['LIBELLE_A_RECHERCHER'], x.DEP), axis=1)

donnees_gendarmerie_nationale_2018_par_com["CODE_INSEE"]=donnees_gendarmerie_nationale_2018_par_com["COMMUNES"].str.replace(r".*,","")
donnees_gendarmerie_nationale_2018_par_com["CODE_INSEE"]=donnees_gendarmerie_nationale_2018_par_com["CODE_INSEE"].str.strip()
donnees_gendarmerie_nationale_2018_par_com["IN_LOC_GN"]=donnees_gendarmerie_nationale_2018_par_com["CODE_INSEE"].isin(donnees_localisation_gn["code_commune_insee"])
ll=list(donnees_gendarmerie_nationale_2018_par_com[~donnees_gendarmerie_nationale_2018_par_com["IN_LOC_GN"]]["LIBELLE_A_RECHERCHER"])

# Reprise manuelle

# '06029'	CANNES   = 601 chemin de Campane 06250 Mougins
# '10325'	ROSIERES PRES TROYES = 58 avenue du 28-Août-1944 10270 Lusigny-sur-Barse
# '11069'	CARCASSONNE = 2 rue Paul-Lacombe 11000 Carcassonne
# '13047'	ISTRES = 541 chemin des Pinèdes 13680 Lançon-Provence
# '17300'	LA ROCHELLE = Rue du Champ Pinson 17137 Nieul-sur-Mer
# '25388'	MONTBELIARD = 17 impasse du Chênois 25460 Étupes
# '28218'	LUCE = 2 rue Jean-Monnet 28000 Chartres
# '2B033'	BASTIA = Place de la Mairie Erbalunga 20222 Brando
# '33009'	ARCACHON = 28 route de Bordeaux Petit Piquey 33950 Lège-Cap-Ferret
# '33065'	BOULIAC = Quartier Béteille 33270 Bouliac
# '38185'	GRENOBLE = 24, avenue du Vercors 38170 Seyssinet-Pariset
# '40192'	MONT DE MARSAN = Caserne du Baradé 50, rue Pierre Benoît 40000 Mont-de-Marsan
# '41018'	BLOIS = 1, rue Paul Valéry 41350 Vineuil
# '44143'	REZE = 23, rue Gutemberg 44340 Bouguenais
# '45208'	MONTARGIS = 2, route de Mirebeau 45210 Ferrières-en-Gâtinais
# '47001'	AGEN = 15 Rue Valence 47000 Agen
# '49099'	CHOLET = 3, allée de Toscane 49450 Saint-Macaire-en-Mauges
# '49328'	SAUMUR = 1, rue Lespagnol de la Plante 49650 Allonnes
# '50129'	CHERBOURG EN COTENTIN = 28, rue d'Aurigny 50440 Beaumont-Hague
# '52448'	ST DIZIER = 13, avenue de la République 52100 Saint-Dizier
# '54099'	NANCY = 2, rue de la Grande Ozeraille 54280 Seichamps
# '54395'	VAL DE BRIEY = 3, place Jean Jaurès 54750 Trieux
# '56121'	LORIENT = 32, rue Du Couëdic 56100 Lorient
# '56260'	VANNES = Caserne Guillaudot 2 Place de la Libération 56000 Vannes
# '57227'	FORBACH = 10 Rue du Petit Bois 57460 Behren-lès-Forbach
# '57672'	THIONVILLE = 3, rue de la Gendarmerie 57330 Hettange-Grande
# '59178'	DOUAI = Caserne Conroux 240, rue des Wetz 59500 Douai
# '59183'	DUNKERQUE HOYMILLE = Chemin Départemental 916 A 59492 Hoymille
# '60159'	COMPIEGNE = 2, avenue Thiers 60200 Compiègne
# '62193'	CALAIS = 528, avenue Matisse 62730 Marck
# '62765'	ST OMER = Tatinghem 1 Allée du Centenaire 62500 Saint-Martin-lez-Tatinghem
# '69029'	BRON = 19, rue de la République 69740 Genas
# '69091'	GIVORS = 1, rue Louis Guillaumond 69440 Mornant
# '69123'	LYON = Caserne Général de Division Delfosse 2 Rue Bichat 69002 Lyon 02
# '71076'	CHALON SUR SAONE = 2, rue Lucie Aubrac 71880 Châtenoy-le-Royal
# '77186'	FONTAINEBLEAU = 27, rue des Bois 77300 Fontainebleau
# '82121'	MONTAUBAN = 19, avenue André Bonnet 82700 Montech
# '84031'	CARPENTRAS = 162, avenue John Kennedy 84200 Carpentras
# '85194'	LES SABLES D OLONNE = 130, rue du 8-Mai-1945 85440 Talmont-Saint-Hilaire
# '86066'	CHATELLERAULT = 5, boulevard Aristide Briand 86100 Châtellerault
# '97120'	POINTE A PITRE = 1 Impasse Joas Section Castaing 97180 Sainte-Anne
# '97416'	ST PIERRE = 20 Avenue Principale 97450 Saint-Louis


<>:11: DeprecationWarning: invalid escape sequence \ 
<>:11: DeprecationWarning: invalid escape sequence \ 
/tmp/ipykernel_1010/1328900203.py:11: DeprecationWarning: invalid escape sequence \ 
  donnees_gendarmerie_nationale_2018_par_com["LIBELLE_A_RECHERCHER"]=donnees_gendarmerie_nationale_2018_par_com['Libellé index \ CGD'].str.replace('CGD','')


[' CANNES', ' ROSIERES PRES TROYES', ' CARCASSONNE', ' ISTRES', ' LA ROCHELLE', ' MONTBELIARD', ' LUCE', ' BASTIA', ' ARCACHON', ' BOULIAC', ' GRENOBLE', ' MONT DE MARSAN', ' BLOIS', ' REZE', ' MONTARGIS', ' AGEN', ' CHOLET', ' SAUMUR', ' CHERBOURG EN COTENTIN', ' ST DIZIER', ' NANCY', ' VAL DE BRIEY', ' LORIENT', ' VANNES', ' FORBACH', ' THIONVILLE', ' DOUAI', ' DUNKERQUE HOYMILLE', ' COMPIEGNE', ' CALAIS', ' ST OMER', ' BRON', ' GIVORS', ' LYON', ' CHALON SUR SAONE', ' FONTAINEBLEAU', ' MONTAUBAN', ' CARPENTRAS', ' LES SABLES D OLONNE', ' CHATELLERAULT', ' POINTE A PITRE', ' ST PIERRE']


/tmp/ipykernel_1010/1328900203.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  donnees_gendarmerie_nationale_2018_par_com["CODE_INSEE"]=donnees_gendarmerie_nationale_2018_par_com["COMMUNES"].str.replace(r".*,","")


In [ ]:
# url_demarche_admin="https://demarchesadministratives.fr/gendarmerie/"
# url_annuaire_admin="https://www.annuaire-administration.com/gendarmerie/"

# r=requests.get(url_annuaire_admin)
# #donnees_localisation_gn[donnees_localisation_gn.adresse_geographique.str.contains("carcassonne",case=False)]
# #donnees_localisation_gn[donnees_localisation_gn.service.str.contains("carcassonne",case=False)]
# #donnees_localisation_gn[donnees_localisation_gn.departement=='11']["url"]

# soup = BeautifulSoup(r.text, 'html.parser')
# liste_url=[a["href"] for a in soup.find_all(attrs={"style": "color:#000000;"})]


In [498]:
#donnees_gendarmerie_nationale_2018_par_com
gendarmerie_xy=donnees_localisation_gn[['code_commune_insee',
       'geocodage_x_GPS', 'geocodage_y_GPS']]
gendarmerie_xy=gendarmerie_xy[gendarmerie_xy["code_commune_insee"].str[:2]!="98"]
# on prend les lignes avec geocodage_GPS
gendarmerie_xy=gendarmerie_xy[~gendarmerie_xy["geocodage_x_GPS"].isna()]
gendarmerie_xy=gendarmerie_xy[~gendarmerie_xy["geocodage_y_GPS"].isna()]
gendarmerie_xy=gendarmerie_xy.set_index("code_commune_insee")

# !!!!! il existe des communes où il y a plusieurs gendarmeries => par hypothèse de parcimonie, on n'en garde une au hasard "first" !!!!!
gendarmerie_xy=gendarmerie_xy[~gendarmerie_xy.index.duplicated(keep="first")]

donnees_gendarmerie_nationale_2018_par_com_geoloc=donnees_gendarmerie_nationale_2018_par_com.reset_index()
donnees_gendarmerie_nationale_2018_par_com_geoloc=donnees_gendarmerie_nationale_2018_par_com_geoloc.set_index("CODE_INSEE")
donnees_gendarmerie_nationale_2018_par_com_geoloc=donnees_gendarmerie_nationale_2018_par_com_geoloc.merge(gendarmerie_xy,how='left',right_index=True,left_index=True)
donnees_gendarmerie_nationale_2018_par_com_geoloc[~donnees_gendarmerie_nationale_2018_par_com_geoloc.IN_LOC_GN].index


Index(['06029', '10325', '11069', '13047', '17300', '25388', '28218', '2B033',
       '33009', '33065', '38185', '40192', '41018', '44143', '45208', '47001',
       '49099', '49328', '50129', '52448', '54099', '54395', '56121', '56260',
       '57227', '57672', '59178', '59183', '60159', '62193', '62765', '69029',
       '69091', '69123', '71076', '77186', '82121', '84031', '85194', '86066',
       '97120', '97416'],
      dtype='object')

In [502]:
donnees_gendarmerie_nationale_2018_par_com_geoloc["ADRESSE"]=""
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="06029","ADRESSE"]=" 601 chemin de Campane 06250 Mougins"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="10325","ADRESSE"]=" 58 avenue du 28-Août-1944 10270 Lusigny-sur-Barse"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="11069","ADRESSE"]=" 2 rue Paul-Lacombe 11000 Carcassonne"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="13047","ADRESSE"]=" 541 chemin des Pinèdes 13680 Lançon-Provence"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="17300","ADRESSE"]=" Rue du Champ Pinson 17137 Nieul-sur-Mer"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="25388","ADRESSE"]=" 17 impasse du Chênois 25460 Étupes"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="28218","ADRESSE"]=" 2 rue Jean-Monnet 28000 Chartres"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="2B033","ADRESSE"]=" Place de la Mairie Erbalunga 20222 Brando"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="33009","ADRESSE"]=" 28 route de Bordeaux Petit Piquey 33950 Lège-Cap-Ferret"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="33065","ADRESSE"]=" Quartier Béteille 33270 Bouliac"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="38185","ADRESSE"]=" 24, avenue du Vercors 38170 Seyssinet-Pariset"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="40192","ADRESSE"]=" Caserne du Baradé 50, rue Pierre Benoît 40000 Mont-de-Marsan"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="41018","ADRESSE"]=" 1, rue Paul Valéry 41350 Vineuil"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="44143","ADRESSE"]=" 23, rue Gutemberg 44340 Bouguenais"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="45208","ADRESSE"]=" 2, route de Mirebeau 45210 Ferrières-en-Gâtinais"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="47001","ADRESSE"]=" 15 Rue Valence 47000 Agen"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="49099","ADRESSE"]=" 3, allée de Toscane 49450 Saint-Macaire-en-Mauges"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="49328","ADRESSE"]=" 1, rue Lespagnol de la Plante 49650 Allonnes"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="50129","ADRESSE"]=" 28, rue d'Aurigny 50440 Beaumont-Hague"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="52448","ADRESSE"]=" 13, avenue de la République 52100 Saint-Dizier"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="54099","ADRESSE"]=" 2, rue de la Grande Ozeraille 54280 Seichamps"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="54395","ADRESSE"]=" 3, place Jean Jaurès 54750 Trieux"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="56121","ADRESSE"]=" 32, rue Du Couëdic 56100 Lorient"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="56260","ADRESSE"]=" Caserne Guillaudot 2 Place de la Libération 56000 Vannes"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="57227","ADRESSE"]=" 10 Rue du Petit Bois 57460 Behren-lès-Forbach"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="57672","ADRESSE"]=" 3, rue de la Gendarmerie 57330 Hettange-Grande"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="59178","ADRESSE"]=" Caserne Conroux 240, rue des Wetz 59500 Douai"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="59183","ADRESSE"]=" Chemin Départemental 916 A 59492 Hoymille"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="60159","ADRESSE"]=" 2, avenue Thiers 60200 Compiègne"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="62193","ADRESSE"]=" 528, avenue Matisse 62730 Marck"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="62765","ADRESSE"]=" Tatinghem 1 Allée du Centenaire 62500 Saint-Martin-lez-Tatinghem"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="69029","ADRESSE"]=" 19, rue de la République 69740 Genas"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="69091","ADRESSE"]=" 1, rue Louis Guillaumond 69440 Mornant"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="69123","ADRESSE"]=" Caserne Général de Division Delfosse 2 Rue Bichat 69002 Lyon 02"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="71076","ADRESSE"]=" 2, rue Lucie Aubrac 71880 Châtenoy-le-Royal"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="77186","ADRESSE"]=" 27, rue des Bois 77300 Fontainebleau"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="82121","ADRESSE"]=" 19, avenue André Bonnet 82700 Montech"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="84031","ADRESSE"]=" 162, avenue John Kennedy 84200 Carpentras"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="85194","ADRESSE"]=" 130, rue du 8-Mai-1945 85440 Talmont-Saint-Hilaire"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="86066","ADRESSE"]=" 5, boulevard Aristide Briand 86100 Châtellerault"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="97120","ADRESSE"]=" 1 Impasse Joas Section Castaing 97180 Sainte-Anne"
donnees_gendarmerie_nationale_2018_par_com_geoloc.loc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="97416","ADRESSE"]=" 20 Avenue Principale 97450 Saint-Louis"
donnees_gendarmerie_nationale_2018_par_com_geoloc[~donnees_gendarmerie_nationale_2018_par_com_geoloc.IN_LOC_GN]
# '10325'	ROSIERES PRES TROYES = 58 avenue du 28-Août-1944 10270 Lusigny-sur-Barse
# '11069'	CARCASSONNE = 2 rue Paul-Lacombe 11000 Carcassonne
# '13047'	ISTRES = 541 chemin des Pinèdes 13680 Lançon-Provence
# '17300'	LA ROCHELLE = Rue du Champ Pinson 17137 Nieul-sur-Mer
# '25388'	MONTBELIARD = 17 impasse du Chênois 25460 Étupes
# '28218'	LUCE = 2 rue Jean-Monnet 28000 Chartres
# '2B033'	BASTIA = Place de la Mairie Erbalunga 20222 Brando
# '33009'	ARCACHON = 28 route de Bordeaux Petit Piquey 33950 Lège-Cap-Ferret
# '33065'	BOULIAC = Quartier Béteille 33270 Bouliac
# '38185'	GRENOBLE = 24, avenue du Vercors 38170 Seyssinet-Pariset
# '40192'	MONT DE MARSAN = Caserne du Baradé 50, rue Pierre Benoît 40000 Mont-de-Marsan
# '41018'	BLOIS = 1, rue Paul Valéry 41350 Vineuil
# '44143'	REZE = 23, rue Gutemberg 44340 Bouguenais
# '45208'	MONTARGIS = 2, route de Mirebeau 45210 Ferrières-en-Gâtinais
# '47001'	AGEN = 15 Rue Valence 47000 Agen
# '49099'	CHOLET = 3, allée de Toscane 49450 Saint-Macaire-en-Mauges
# '49328'	SAUMUR = 1, rue Lespagnol de la Plante 49650 Allonnes
# '50129'	CHERBOURG EN COTENTIN = 28, rue d'Aurigny 50440 Beaumont-Hague
# '52448'	ST DIZIER = 13, avenue de la République 52100 Saint-Dizier
# '54099'	NANCY = 2, rue de la Grande Ozeraille 54280 Seichamps
# '54395'	VAL DE BRIEY = 3, place Jean Jaurès 54750 Trieux
# '56121'	LORIENT = 32, rue Du Couëdic 56100 Lorient
# '56260'	VANNES = Caserne Guillaudot 2 Place de la Libération 56000 Vannes
# '57227'	FORBACH = 10 Rue du Petit Bois 57460 Behren-lès-Forbach
# '57672'	THIONVILLE = 3, rue de la Gendarmerie 57330 Hettange-Grande
# '59178'	DOUAI = Caserne Conroux 240, rue des Wetz 59500 Douai
# '59183'	DUNKERQUE HOYMILLE = Chemin Départemental 916 A 59492 Hoymille
# '60159'	COMPIEGNE = 2, avenue Thiers 60200 Compiègne
# '62193'	CALAIS = 528, avenue Matisse 62730 Marck
# '62765'	ST OMER = Tatinghem 1 Allée du Centenaire 62500 Saint-Martin-lez-Tatinghem
# '69029'	BRON = 19, rue de la République 69740 Genas
# '69091'	GIVORS = 1, rue Louis Guillaumond 69440 Mornant
# '69123'	LYON = Caserne Général de Division Delfosse 2 Rue Bichat 69002 Lyon 02
# '71076'	CHALON SUR SAONE = 2, rue Lucie Aubrac 71880 Châtenoy-le-Royal
# '77186'	FONTAINEBLEAU = 27, rue des Bois 77300 Fontainebleau
# '82121'	MONTAUBAN = 19, avenue André Bonnet 82700 Montech
# '84031'	CARPENTRAS = 162, avenue John Kennedy 84200 Carpentras
# '85194'	LES SABLES D OLONNE = 130, rue du 8-Mai-1945 85440 Talmont-Saint-Hilaire
# '86066'	CHATELLERAULT = 5, boulevard Aristide Briand 86100 Châtellerault
# '97120'	POINTE A PITRE = 1 Impasse Joas Section Castaing 97180 Sainte-Anne
# '97416'	ST PIERRE = 20 Avenue Principale 97450 Saint-Louis

/tmp/ipykernel_1010/1102074302.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  donnees_gendarmerie_nationale_2018_par_com_geoloc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="10325"]["ADRESSE"]=" 58 avenue du 28-Août-1944 10270 Lusigny-sur-Barse"
/tmp/ipykernel_1010/1102074302.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  donnees_gendarmerie_nationale_2018_par_com_geoloc[donnees_gendarmerie_nationale_2018_par_com_geoloc.index=="11069"]["ADRESSE"]=" 2 rue Paul-Lacombe 11000 Carcassonne"

,index,Libellé index \ CGD,Règlements de compte entre malfaireurs,Homicides pour voler et à l'occasion de vols,Homicides pour d'autres motifs,Tentatives d'homicides pour voler et à l'occasion de vols,Tentatives homicides pour d'autres motifs,Coups et blessures volontaires suivis de mort,Autres coups et blessures volontaires criminels ou correctionnels,Prises d'otages à l'occasion de vols,...,Fraudes fiscales,Autres délits économiques et financiers,Autres délits,LIBELLE_A_RECHERCHER,DEP,COMMUNES,IN_LOC_GN,geocodage_x_GPS,geocodage_y_GPS,ADRESSE
06029,06,CGD CANNES,0,0,1,0,5,0,334,0,...,1,8,288,CANNES,06,"CANNES ,06029",False,NaN,NaN,601 chemin de Campane 06250 Mougins
10325,10.2,CGD ROSIERES PRES TROYES,0,0,0,0,1,0,142,0,...,0,2,131,ROSIERES PRES TROYES,10,"ROSIERES PRES TROYES ,10325",False,NaN,NaN,
11069,11,CGD CARCASSONNE,0,0,0,0,4,1,276,0,...,0,1,109,CARCASSONNE,11,"CARCASSONNE ,11069",False,NaN,NaN,
13047,13.3,CGD ISTRES,0,0,1,0,0,0,154,0,...,1,0,123,ISTRES,13,"ISTRES ,13047",False,NaN,NaN,
17300,17.1,CGD LA ROCHELLE,0,0,0,0,4,1,189,0,...,2,2,124,LA ROCHELLE,17,"ROCHELLE ,17300",False,NaN,NaN,
25388,25.1,CGD MONTBELIARD,0,0,1,0,0,0,280,1,...,2,3,193,MONTBELIARD,25,"MONTBELIARD ,25388",False,NaN,NaN,
28218,28.2,CGD LUCE,0,0,0,0,4,0,185,1,...,0,6,147,LUCE,28,"LUCE ,28218",False,NaN,NaN,
2B033,2B,CGD BASTIA,0,0,1,0,4,0,115,0,...,0,0,87,BASTIA,2B,"BASTIA ,2B033",False,NaN,NaN,
33009,33,CGD ARCACHON,0,0,2,0,4,0,258,0,...,0,3,130,ARCACHON,33,"ARCACHON ,33009",False,NaN,NaN,
33065,33.2,CGD BOULIAC,0,0,0,2,3,1,381,0,...,4,1,213,BOULIAC,33,"BOULIAC ,33065",False,NaN,NaN,


In [496]:
#50 appels par seconde autorisés
strinput='41 Boulevard Louis Dupuy 01100 OYONNAX'
url_api="https://api-adresse.data.gouv.fr/search/?q="
response=requests.get(url_api+strinput)
print(response.json())

{'type': 'FeatureCollection', 'version': 'draft', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [5.650245, 46.257487]}, 'properties': {'label': '41 Boulevard Louis Dupuy 01100 Oyonnax', 'score': 0.9675327272727272, 'housenumber': '41', 'id': '01283_0940_00041', 'name': '41 Boulevard Louis Dupuy', 'postcode': '01100', 'citycode': '01283', 'x': 904112.23, 'y': 6576493.71, 'city': 'Oyonnax', 'context': '01, Ain, Auvergne-Rhône-Alpes', 'type': 'housenumber', 'importance': 0.64286, 'street': 'Boulevard Louis Dupuy'}}], 'attribution': 'BAN', 'licence': 'ETALAB-2.0', 'query': '41 Boulevard Louis Dupuy 01100 OYONNAX', 'limit': 5}


## Fusion des données de la Police Nationale et de la Gendarmerie Nationale 

In [ ]:
# merged_crimes_delits_2018_par_dep = donnees_police_nationale_2018_par_dep.merge(donnees_gendarmerie_nationale_2018_par_dep, on="departement", how='left')
# merged_crimes_delits_2018_par_dep

## 5 - Téléchargement des revenus localisés

In [ ]:
URL_FILO_ZIP_2018="https://www.insee.fr/fr/statistiques/fichier/5009236/base-cc-filosofi-2018_CSV_geo2021.zip"
#Télécharger le zip de l'URL
r=requests.get(URL_FILO_ZIP_2018)

open("base-cc-filosofi-2018_CSV_geo2021.zip", 'wb').write(r.content)
    
    
with ZipFile("base-cc-filosofi-2018_CSV_geo2021.zip",'r') as myzip:
    data = myzip.open("cc_filosofi_2018_COM-geo2021.CSV")
    
donnees_filo_communes_2018=pd.read_csv(data,sep=';')       

In [ ]:
donnees_filo_communes_2018

## 6 - Téléchargement des populations légales

In [ ]:
#Téléchargement des populations légales du millénisme 2018 
URL_POP_LEG="https://www.insee.fr/fr/statistiques/fichier/4989724/ensemble.zip"

#Télécharger le zip de l'URL
r=requests.get(URL_POP_LEG)

open("ensemble.zip", 'wb').write(r.content)
    
    
with ZipFile("ensemble.zip",'r') as myzip:
    data = myzip.open("Communes.csv")

donnees_pop_leg_18=pd.read_csv(data,sep=';',dtype=str)
donnees_pop_leg_18["CODE_INSEE"]=donnees_pop_leg_18["CODREG"]+donnees_pop_leg_18["CODCOM"]
donnees_pop_leg_18=donnees_pop_leg_18.set_index("CODE_INSEE")


In [ ]:
donnees_pop_leg_18.sample(10)

In [ ]:
donnees_pop_leg_18.describe()

## 7 - Téléchargement des données d'emplois localisées

In [ ]:
URL_TAUX_CHOMAGE_15_24_PAR_COM="https://www.observatoire-des-territoires.gouv.fr/outils/cartographie-interactive/api/v1/functions/GC_API_download.php?type=stat&nivgeo=com2021&dataset=indic_sex_rp&indic=tx_chom1524"


donnees_chomage_15_24_par_com=pd.read_excel(URL_TAUX_CHOMAGE_15_24_PAR_COM, sheet_name='Data',skiprows=4)

donnees_chomage_15_24_par_com
# donnees_chomage_par_departement_series_longues=donnees_chomage_par_departement_series_longues.iloc[:100]
# donnees_chomage_par_departement_series_longues=donnees_chomage_par_departement_series_longues.set_index("Code")
# donnees_chomage_par_departement_series_longues.index=donnees_chomage_par_departement_series_longues.index.astype(str)

In [ ]:
donnees_chomage_15_24_par_com_travail=donnees_chomage_15_24_par_com[donnees_chomage_15_24_par_com["an"]==2018]
donnees_chomage_15_24_par_com_travail=donnees_chomage_15_24_par_com_travail[donnees_chomage_15_24_par_com_travail["sexe"]=="T"]
donnees_chomage_15_24_par_com_travail

## 8 - Fusion des données

In [ ]:
# donnees_2018=merged_crimes_delits_2018_par_dep.merge(donnees_filo_departement_2018,left_index=True,right_index=True)
# donnees_2018=donnees_2018.merge(donnees_pop_leg_18,left_index=True,right_index=True)
# donnees_2018=donnees_2018.merge(donnees_sirene_fr_agregees_par_departement,left_index=True,right_index=True)
# donnees_2018=donnees_2018.merge(donnees_chomage_par_departement_series_longues,left_index=True,right_index=True)
# donnees_2018=donnees_2018.merge(donnees_nb_pn_gn_dep,left_index=True,right_index=True)
# donnees_2018=donnees_2018[['REG', 'Libellé', 'Total des crimes et délits relevés par la Police Nationale', 'Total des crimes et délits relevés par la Gendarmerie Nationale', 'MED18' , 'TP6018', 'D118', 'D918', 'RD18', 'PTOT','T1_2018', 'NbBoucheriesParDep', 'Nb_PN_GN_dep_100k_hab']]
# donnees_2018

* On se restreint au champ métropolitain car les données d'outre mer ne sont pas exhaustives sur toutes nos sources 

In [ ]:
# donnees_2018 = donnees_2018[donnees_2018.index.str[:2]!='97']
# donnees_2018
# donnees_2018.describe()

* On fait l'hypothèse d'unicité du dépôt de plainte ou de signalement en gardarmerie ou bien en commissariat, ce qui permet de sommer les crimes et délits au niveau communal

In [ ]:
# donnees_2018['Total des crimes et délits relevés par la PN et GN'] = donnees_2018[['Total des crimes et délits relevés par la Police Nationale', 'Total des crimes et délits relevés par la Gendarmerie Nationale']].sum(axis=1)

In [ ]:
# donnees_2018 = donnees_2018.rename(columns={"Total des crimes et délits relevés par la Police Nationale": "Crim_Del_PN", "Total des crimes et délits relevés par la Gendarmerie Nationale": "Crim_Del_GN", "Total des crimes et délits relevés par la PN et GN": "Crim_Del_PN_GN"})
# donnees_2018

## 9 - On créé une base de données rapportant le nombre de crimes et délits, et le nombre de boucheries pour 100 000 habitants 


In [ ]:
# donnees_2018_hab = donnees_2018
# donnees_2018_hab['Nb_Boucherie_dep_hab']=round(donnees_2018_hab['NbBoucheriesParDep']/donnees_2018_hab['PTOT']*100000)
# donnees_2018_hab['Crim_Del_GN_hab']=round(donnees_2018_hab['Crim_Del_GN']/donnees_2018_hab['PTOT']*100000)
# donnees_2018_hab['Crim_Del_PN_hab']=round(donnees_2018_hab['Crim_Del_PN']/donnees_2018_hab['PTOT']*100000)
# donnees_2018_hab['Crim_Del_PN_GN_hab']=round(donnees_2018_hab['Crim_Del_PN_GN']/donnees_2018_hab['PTOT']*100000)
# donnees_2018_hab.drop(['Crim_Del_PN', 'Crim_Del_GN', 'PTOT', 'NbBoucheriesParDep'], axis=1, inplace = True)
# donnees_2018_hab


In [ ]:
# %store donnees_2018
# %store donnees_2018_hab

## 10 - Tentative de récupération d'un proxy d'un sentiment de sécurité au niveau communal


In [ ]:
# import re

# import time
# headers = {
#     'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
# }

# url="https://www.ville-ideale.fr/villespardepts.php"
# url_cherche="https://www.ville-ideale.fr/scripts/cherche.php"
# base_url="https://www.ville-ideale.fr"
# s = requests.session()
# r=s.get(url,headers=headers)
# print(r)
# print(r.text)
# time.sleep(1)

# liste_departement=re.findall(r'affdept\(\'(\d+)\',',r.text)
# for _, numdep in enumerate(liste_departement):
#     r=s.post(url_cherche,data={'dept': numdep},headers=headers)
#     print(r)
#     soup = BeautifulSoup(r.text, 'html.parser')
#     for link in soup.find_all('a'):
#         time.sleep(1)
#         print(link.get('href'))
#         r=s.get(base_url+link.get('href'),headers=headers)
#         print(r)
#         soup = BeautifulSoup(r.text, 'html.parser')
#         print(soup.find(id="tablonotes").findAll("tr"))
#         print(type(soup.find(id="tablonotes").findAll("tr")))
#         liste_tr_securite=[tr.text for tr in list(soup.find(id="tablonotes").findAll("tr")) if "Sécurité" in tr.text ]
#         if liste_tr_securite:
#             note_securite=re.findall(r'\d+', liste_tr_securite[0])
#             print('.'.join(note_securite))
#         break
#     break

In [ ]:
# !pip install lxml
# from lxml.html import fromstring
# import requests
# from itertools import cycle
# import traceback
# def get_proxies():
#     url = 'https://free-proxy-list.net/'
#     response = requests.get(url)
#     parser = fromstring(response.text)
#     proxies = set()
#     for i in parser.xpath('//tbody/tr'):
#         if i.xpath('.//td[7][contains(text(),"yes")]') and  i.xpath('.//td[5][contains(text(),"elite proxy")]'):
#             proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
#             proxies.add(proxy)
#     return proxies

# proxies = get_proxies()
# print(proxies)
# proxy_pool = cycle(proxies)
# url = 'https://www.ville-ideale.fr'

# bool_continue=True
# i=0
# while bool_continue and i <10:
# #Get a proxy from the pool
#     i+=1
#     proxy = next(proxy_pool)
#     print("Request #%d"%i)
#     try:
#         s = requests.Session()
#         s.proxies = {"http": proxy, "https": proxy}
#         response = s.get(url)
#         print(response.json())
#         bool_continue=False
#     except:
#         #Most free proxies will often get connection errors. You will have retry the entire request using another proxy to work. 
#         #We will just skip retries as its beyond the scope of this tutorial and we are only downloading a single url 
#         print("Skipping. Connnection error")